check if there are mismatches between the datas of the 2 datasets

### read files

In [1]:
import pandas as pd

In [2]:
nodes = pd.read_json('nodesFinal.json', encoding='utf-8')

In [3]:
edges = pd.read_json('edgesFinal.json', encoding='utf-8')

### check match edges sources and targets in nodes

In [4]:
allSourcesAndTargets = set(list(edges["source"].unique()) + list(edges["target"].unique()))
len(allSourcesAndTargets)

3417

In [5]:
allIds = set(nodes["id"]) #checked and there are no repetitions
len(allIds)

2816

why is there a difference? did i remove data that i should have not?

In [6]:
nodesOldIds = pd.read_json("nodes.json")["id"]
len(nodesOldIds)

3428

yes i removed the data which doesnt have a type but there is still a mismatch between the old nodes and the number of sources and targets in the edges (?)

In [7]:
nodeFinalEdgesFinalIntersections = allSourcesAndTargets & allIds
"len intersection edge final, node final", len(nodeFinalEdgesFinalIntersections), "len intersection and len nodes matches:", len(nodeFinalEdgesFinalIntersections) == len(allIds)

('len intersection edge final, node final',
 2816,
 'len intersection and len nodes matches:',
 True)

In [8]:
missingKeys = allSourcesAndTargets-allIds
len(missingKeys), missingKeys

(601,
 {'07',
  '076',
  '13 yearsâ',
  '13d79d632036',
  '17 Sept',
  '189',
  '2030â',
  '2035/03',
  '2036-06',
  '2037/09',
  '20370602035941',
  '2038650116',
  '22 Raptor',
  '255â\x80\x93273',
  '3 Ltd. Liability Co',
  '4â\x80\x89Ã\x97â\x80\x8915â',
  '6 Ray LLC Enterprises',
  '80 Percent Arms',
  'AND fisheryâ',
  'Agua TransiciÃ³n EcolÃ³gica',
  'Aicrowe Biotech Co',
  'AlaÄ\x9dum KanuuxÌ',
  'Aleut Community St',
  'All',
  'Alonso-FernÃ',
  'Alpha Brokersa',
  'America',
  'American',
  'Amorim Flooring',
  'Amstel Inc',
  'Ancla Azul Company Solutions',
  'Ancla del Mar Ltd. Corporation',
  'Andhra Pradesh  Investment Dry dock',
  'Andhra Pradesh  S.A. de C.V.',
  'Anthony Sailboat Co Shipping',
  'Aqua Ambassadors Limited Liability Company',
  'Aquacultureâ',
  'Arctic',
  'Armed Robberyâ',
  'Artemovâ\x80\x99s Ava Petroleum',
  'Arunachal Pradesh Sp Marine mist',
  'Asia Seafood',
  'Atlantoâ\x80\x93Scandian',
  'Attorneysâ',
  'Auraâ\x80\x99s',
  'BPâ\x80\x99s',
  'Bar

so we can proceed with merging the two datasets, even tho some data will be missing (see below for the total amount and list of missing ones) 

### merging datasets

In [9]:
nodesIdDict = nodes.set_index('id').T.to_dict()
len(nodes), len(nodesIdDict)

(2816, 2816)

In [10]:
nodesIdDict.keys()

dict_keys(['Spanish Shrimp  Carriers', '12744', '143129355', '7775', '1017141', '2591586', '185040354', 'Faroe Islands Shrimp Shark', '341411', '21323516', '8561', 'William Bradley', 'Joseph Griffin', '853949688', '650512', '8', '128692', '172711', 'Jacob Caldwell', '14253815', '4369922c-ce1d-40ff-904c-1f6a3000bf6b', 'Saltwater Solitude GmbH & Co. KG', '506', '4937447', '2079436', 'Ann Daniels', 'Stephanie Mccormick MD', 'Lisa Clay', '589', 'Elizabeth Welch', 'da0a7bbb-5bd1-42aa-bcc0-b8923d5c15d4', 'Eugene Ochoa', '967900443', 'Lisa Jimenez', '1', '445a6124-4650-4e5a-83bb-a0e0b4040ee9', 'Martha Robbins', 'Don Hopkins', 'Joseph Gray', '4', '4975', '7282284', '0', 'Sean Vasquez', '196159', '755', '860067396', '664738735', '290834957', 'Timothy Reeves', '457009514', '6043477', '526269', 'Jason Cameron', '77b46bc7-7e31-41a0-973c-e633011d21b0', '7', '35', '6406040', 'Jamie Knox', 'Kristin Graham', '99575926', '72371354', '55', '766649', '955425', '132', '8701038', '9067', '750975', '3404599

In [11]:
edges

,type,weight,source,target
0,ownership,0.900140,Spanish Shrimp Carriers,12744
1,partnership,0.845897,Spanish Shrimp Carriers,21323516
2,partnership,0.964876,Spanish Shrimp Carriers,290834957
3,ownership,0.964213,Spanish Shrimp Carriers,3506021
4,family_relationship,0.823486,Spanish Shrimp Carriers,Conventionâ
...,...,...,...,...
11064,family_relationship,0.817772,Ocano del Este SE,Mary Clements
11065,partnership,0.873537,Norwegian Shrimp N.V. -,67261634
11066,partnership,0.849333,Norwegian Shrimp N.V. -,344
11067,membership,0.814217,Malabar Sea Catch Sp,672


In [23]:
import pandas as pd

# Assuming nodesIdDict and edges DataFrame are defined elsewhere

def add_infos_to_row(row, nodes_id_dict):
    source_info = nodes_id_dict.get(row["source"], {"type": None, "country": None})
    target_info = nodes_id_dict.get(row["target"], {"type": None, "country": None})

    return pd.Series({
        'typeOfLink': row["type"],
        'weight': row["weight"],
        'source': row["source"],
        'sourceType': source_info["type"],
        'target': row["target"],
        'targetType': target_info["type"],
        'sourceCountry': source_info["country"],
        'targetCountry': target_info["country"]
    })

def process_dataset(edges, nodes_id_dict):
    new_columns = [
        'typeOfLink', 'weight',
        'source', 'sourceType', 
        'target', 'targetType', 
        'sourceCountry', 'targetCountry'
    ]

    processed_rows = edges.apply(add_infos_to_row, axis=1, nodes_id_dict=nodes_id_dict)
    return pd.DataFrame(processed_rows, columns=new_columns)

# Assuming edges DataFrame and nodesIdDict dictionary are defined
mergedDataset = process_dataset(edges, nodesIdDict)


In [25]:
mergedDataset.to_json('mergedDataset.json', orient='records', indent=4)

### Read new merged file

In [46]:
import pandas as pd

In [47]:
df = pd.read_json('mergedDataset.json')

In [48]:
for col in df.columns:
    # Count and print the number of missing values in the column
    print(f'{col}: {df[col].isnull().sum()}')
    
    # Fill missing values with 'Unknown'
    df[col] = df[col].fillna('Unknown')
    
    # Print the number of missing values again to confirm they are filled
    print(f'{col}: {df[col].isnull().sum()}')

typeOfLink: 0
typeOfLink: 0
weight: 0
weight: 0
source: 0
source: 0
sourceType: 2594
sourceType: 0
target: 0
target: 0
targetType: 2769
targetType: 0
sourceCountry: 8066
sourceCountry: 0
targetCountry: 7522
targetCountry: 0


some of the data is missing but i prefer to plot the data that is there instead of ignoring it so i filled the missing values with 'Unknown'

In [49]:
mergedDataset

,typeOfLink,weight,source,sourceType,target,targetType,sourceCountry,targetCountry
0,ownership,0.900140,Spanish Shrimp Carriers,company,12744,organization,Nalakond,None
1,partnership,0.845897,Spanish Shrimp Carriers,company,21323516,organization,Nalakond,None
2,partnership,0.964876,Spanish Shrimp Carriers,company,290834957,organization,Nalakond,None
3,ownership,0.964213,Spanish Shrimp Carriers,company,3506021,organization,Nalakond,None
4,family_relationship,0.823486,Spanish Shrimp Carriers,company,Conventionâ,None,Nalakond,None
...,...,...,...,...,...,...,...,...
11064,family_relationship,0.817772,Ocano del Este SE,None,Mary Clements,person,None,Country_271835
11065,partnership,0.873537,Norwegian Shrimp N.V. -,None,67261634,organization,None,None
11066,partnership,0.849333,Norwegian Shrimp N.V. -,None,344,organization,None,None
11067,membership,0.814217,Malabar Sea Catch Sp,None,672,organization,None,None
